# Project 1, Part 4, Holiday Analytics

University of California, Berkeley
Master of Information and Data Science (MIDS) program
w205 - Fundamentals of Data Engineering

Student: Landon Morin

Year: 2022

Semester: Spring

Section:


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [3]:
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [4]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [5]:
cursor = connection.cursor()

## How do holidays affect the sales, considering both the actual holiday, and the days before and after the holiday? 

## Create an executive summary explaining how holidays have affected sales. 

## You must support your summary with data, in the form of output of queries, data visualization, etc. There is a 1 query minimum.

In [21]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select sa.sale_date,
       to_char(date_trunc('day', sa.sale_date), 'Day') as day_of_week,
       sum(sa.total_amount) as total_sales,
       h.description as holiday 
from sales as sa
    full join holidays as h
        on h.holiday_date = sa.sale_date
group by sa.sale_date, h.holiday_date
order by sa.sale_date, extract(dow from sa.sale_date)

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,sale_date,day_of_week,total_sales,holiday
0,2020-01-01,Wednesday,133776,New Year's Day
1,2020-01-02,Thursday,261120,None
2,2020-01-03,Friday,263076,None
3,2020-01-04,Saturday,400368,None
4,2020-01-05,Sunday,394992,None
...,...,...,...,...
361,2020-12-29,Tuesday,130440,None
362,2020-12-30,Wednesday,263424,None
363,2020-12-31,Thursday,266100,None
364,None,None,<NA>,Thanksgiving


In [27]:
df1 = df.loc[df['holiday'].isna()]
df1

,sale_date,day_of_week,total_sales,holiday
1,2020-01-02,Thursday,261120,None
2,2020-01-03,Friday,263076,None
3,2020-01-04,Saturday,400368,None
4,2020-01-05,Sunday,394992,None
5,2020-01-06,Monday,262020,None
...,...,...,...,...
359,2020-12-27,Sunday,396096,None
360,2020-12-28,Monday,264852,None
361,2020-12-29,Tuesday,130440,None
362,2020-12-30,Wednesday,263424,None


In [29]:
df2 = df.loc[df['holiday'].isna() == False]
df2

,sale_date,day_of_week,total_sales,holiday
0,2020-01-01,Wednesday,133776,New Year's Day
19,2020-01-20,Monday,130740,MLK Day
47,2020-02-17,Monday,135228,President's Day
102,2020-04-12,Sunday,136164,Easter
130,2020-05-10,Sunday,134496,Mother's Day
145,2020-05-25,Monday,132624,Memorial Day
172,2020-06-21,Sunday,135120,Father's Day
185,2020-07-04,Saturday,134952,Independence Day
250,2020-09-07,Monday,130020,Labor Day
315,2020-11-11,Wednesday,271920,Veterans Days
